In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
import os
import shutil
import copy
import matplotlib.pyplot as plt
from   torchvision import transforms
%matplotlib inline

##### 读取数据集

In [2]:
base_dir = r'data/FourWeather/'
train_dir= os.path.join(base_dir,"train")
test_dir = os.path.join(base_dir,"test")

In [3]:
transform = transforms.Compose([
                  transforms.Resize((96, 96)),
                  transforms.ToTensor(),
                  transforms.Normalize(
                      mean=[0.5, 0.5, 0.5],
                      std =[0.5, 0.5, 0.5]
                  )
])

In [4]:
BATCH_SIZE = 8

In [5]:
train_ds = torchvision.datasets.ImageFolder(
    os.path.join(base_dir,"train"),
    transform = transform
)

test_ds  = torchvision.datasets.ImageFolder(
    os.path.join(base_dir,"test"),
    transform = transform
)

train_dl = torch.utils.data.DataLoader(train_ds,batch_size=BATCH_SIZE,shuffle=True)
test_dl  = torch.utils.data.DataLoader(test_ds,batch_size=BATCH_SIZE,shuffle=True)

In [6]:
class_to_id = train_ds.class_to_idx
id_to_class = dict((v,k) for k,v in train_ds.class_to_idx.items())

##### 定义模型

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool  = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.fc1   = nn.Linear(64*10*10, 1024)
        self.fc2   = nn.Linear(1024, 256)
        self.fc3   = nn.Linear(256, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 10 * 10)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
model   = Net()
if torch.cuda.is_available():
    model.to('cuda')
optim   = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

epochs     = 10
train_loss = []
train_acc  = []
test_loss  = []
test_acc   = []

##### 定义fit函数

In [9]:
def fit(epoch, model, trainloader, testloader):
    correct = 0
    total = 0
    running_loss = 0
    model.train()
    for x, y in trainloader:
        if torch.cuda.is_available():
            x, y = x.to('cuda'), y.to('cuda')
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)
            correct += (y_pred == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()
    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc = correct / total
        
        
    test_correct = 0
    test_total = 0
    test_running_loss = 0 
    
    model.eval()
    with torch.no_grad():
        for x, y in testloader:
            if torch.cuda.is_available():
                x, y = x.to('cuda'), y.to('cuda')
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            test_correct += (y_pred == y).sum().item()
            test_total += y.size(0)
            test_running_loss += loss.item()
    
    epoch_test_loss = test_running_loss / len(testloader.dataset)
    epoch_test_acc = test_correct / test_total
    
        
    print('epoch: ', epoch, 
          'loss： ', round(epoch_loss, 3),
          'accuracy:', round(epoch_acc, 3),
          'test_loss： ', round(epoch_test_loss, 3),
          'test_accuracy:', round(epoch_test_acc, 3)
             )
        
    return epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc

##### 训练模型

In [10]:
for epoch in range(epochs):
    epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc = fit(epoch,model,train_dl,test_dl)
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    test_loss.append(epoch_test_loss)
    test_acc.append(epoch_test_acc)

epoch:  0 loss：  0.1 accuracy: 0.639 test_loss：  0.084 test_accuracy: 0.738
epoch:  1 loss：  0.064 accuracy: 0.79 test_loss：  0.08 test_accuracy: 0.8
epoch:  2 loss：  0.057 accuracy: 0.828 test_loss：  0.07 test_accuracy: 0.818
epoch:  3 loss：  0.044 accuracy: 0.858 test_loss：  0.061 test_accuracy: 0.884
epoch:  4 loss：  0.042 accuracy: 0.88 test_loss：  0.078 test_accuracy: 0.836
epoch:  5 loss：  0.036 accuracy: 0.9 test_loss：  0.088 test_accuracy: 0.831
epoch:  6 loss：  0.031 accuracy: 0.901 test_loss：  0.074 test_accuracy: 0.898
epoch:  7 loss：  0.027 accuracy: 0.914 test_loss：  0.081 test_accuracy: 0.871
epoch:  8 loss：  0.02 accuracy: 0.937 test_loss：  0.078 test_accuracy: 0.858
epoch:  9 loss：  0.017 accuracy: 0.942 test_loss：  0.085 test_accuracy: 0.884


#### 模型的保存  (本质 : 保存训练好的模型的参数)

In [11]:
model.state_dict()                                  # 返回一个字典 将模型中可训练参数(权重)和权值返回

OrderedDict([('conv1.weight',
              tensor([[[[ 8.9591e-02,  9.4120e-02, -2.9476e-02],
                        [ 1.3185e-01,  9.4362e-02,  2.0382e-02],
                        [ 1.9112e-01,  2.1662e-01,  1.2812e-01]],
              
                       [[ 1.4434e-01, -7.4186e-02,  9.6240e-03],
                        [-9.4958e-04,  8.4399e-03, -1.4463e-01],
                        [-7.1932e-02, -1.6459e-01,  1.7691e-02]],
              
                       [[-1.6908e-01, -5.5856e-02, -1.1313e-01],
                        [ 9.3552e-02, -1.1250e-01,  2.9621e-02],
                        [ 1.5981e-01,  1.8239e-02,  1.0484e-01]]],
              
              
                      [[[-4.8685e-03, -1.6254e-01, -2.4681e-02],
                        [ 4.0850e-02, -1.5338e-01, -6.6552e-02],
                        [ 1.7566e-01,  2.0099e-01, -9.4423e-03]],
              
                       [[-1.9084e-01, -1.2496e-01,  4.7322e-02],
                        [-1.5655e-02, -1.4825

In [12]:
PATH = "my_model.pth"                               # 习惯上将训练好的模型的权值保存到.pth文件中

In [13]:
torch.save(model.state_dict(),PATH)                 # 将模型的权重保存到指定位置

##### 模型的加载

In [14]:
new_model = Net()                                   # 新建一个模型对象

In [15]:
new_model.load_state_dict(torch.load(PATH))         # 在新模型上加载已有的权重

<All keys matched successfully>

In [16]:
#### 加载模型完毕后的测试
test_correct      = 0
test_total        = 0
test_running_loss = 0 

new_model         = new_model.to("cuda")

model.eval()                                        # 在测试的过程中必须要使用eval()模式继续进行
with torch.no_grad():
    for x, y in test_dl:
        if torch.cuda.is_available():
            x, y = x.to('cuda'), y.to('cuda')
        y_pred = new_model(x)
        loss = loss_fn(y_pred, y)
        y_pred = torch.argmax(y_pred, dim=1)
        test_correct += (y_pred == y).sum().item()
        test_total += y.size(0)
        test_running_loss += loss.item()

epoch_test_loss = test_running_loss / len(test_dl.dataset)
epoch_test_acc = test_correct / test_total
print(round(epoch_test_acc, 3))                     # 计算并输出模型在一个epoch之后在测试数据集上的准确率

0.884


##### 保存最优模型权重

In [17]:
best_model_wts = copy.deepcopy(model.state_dict())  # 使用深拷贝对当前模型权重进行保存

In [18]:
best_acc       = 0                                  # 存储当前模型最优准确率

In [ ]:
for epoch in range(epochs):
    epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc = fit(epoch,model,train_dl,test_dl)
    
    
    if epoch_test_acc  > best_acc:
        best_model_wts = copy.deepcopy(model.state_dict()) # 保存最优模型的参数
        best_acc       = epoch_test_acc                    # 更新最新的正确率
     
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    test_loss.append(epoch_test_loss)
    test_acc.append(epoch_test_acc)
model.load_state_dict(best_model_wts)                      # 将最优权重赋值给当前模型

epoch:  0 loss：  0.009 accuracy: 0.968 test_loss：  0.102 test_accuracy: 0.871
epoch:  1 loss：  0.018 accuracy: 0.958 test_loss：  0.117 test_accuracy: 0.88
epoch:  2 loss：  0.009 accuracy: 0.976 test_loss：  0.087 test_accuracy: 0.889
epoch:  3 loss：  0.007 accuracy: 0.981 test_loss：  0.152 test_accuracy: 0.871
